# M3.1 - Containerization with Docker

**Production Deployment for RAG Systems**

In this notebook:
1. Reality Check: What Docker Solves / Doesn't
2. Project Layout & Minimal FastAPI App
3. Dockerfile Walkthrough
4. docker-compose for Local Dev
5. Image Size & Security Notes
6. Alternatives (VMs, Bare Metal, K8s) Decision Card
7. Troubleshooting

---

## 1. Reality Check: What Docker Actually Does

### What Docker DOES Well

- **Environment consistency**: If it works on your laptop, it works in production (eliminates 60-80% of deployment issues)
- **Horizontal scaling**: Spin up 100 containers from 1 in minutes
- **Dependency management**: Everything defined in code, version-controlled

### What Docker DOESN'T Do

- **Security**: Vulnerable app in container = still vulnerable. Container isolation ≠ security solution
- **Performance**: Adds 10-20% I/O overhead. Not ideal for ultra-low latency (<5ms)
- **Simplicity**: Adds networking complexity. Debugging requires new skills

### The Trade-offs

| Gain | Cost |
|------|------|
| Portability | Direct metal performance |
| Consistency | Deployment complexity |
| Infrastructure as code | Learning curve (4-6 hours) |

**Financial Cost:**
- Docker Desktop: Free for small teams, $5-9/user/month for companies 250+
- Image registries: $5-50/month
- Time investment: 4-6 hours learning curve

In [ ]:
# Check if Docker is available
import subprocess
import sys

def check_docker():
    try:
        result = subprocess.run(
            ['docker', '--version'],
            capture_output=True,
            text=True,
            timeout=5
        )
        if result.returncode == 0:
            print(f"✓ {result.stdout.strip()}")
            return True
        else:
            print("✗ Docker not found")
            return False
    except Exception as e:
        print(f"✗ Docker check failed: {e}")
        print("\nTo install: Visit https://docs.docker.com/get-docker/")
        return False

# Expected:
# ✓ Docker version 24.0.0 (or similar)
check_docker()

## 2. Project Layout & Minimal FastAPI App

### Project Structure

```
rag-production/
├── app.py                 # FastAPI application
├── requirements.txt       # Python dependencies
├── .env.example          # Environment template
├── docker/
│   ├── Dockerfile        # Multi-stage container blueprint
│   ├── docker-compose.yml # Multi-container orchestration
│   └── .dockerignore     # Build exclusions
├── notebooks/
│   └── M3_1_Containerization_with_Docker.ipynb
├── tests/
│   ├── test_smoke.py     # FastAPI TestClient tests
│   └── test_docker_sanity.py # Docker setup tests
└── scripts/
    └── run_local.ps1     # Local development script
```

### Key Files Created

All files follow TVH's standard per-module layout:

1. **app.py** - FastAPI app with health endpoint, CLI interface
2. **docker/Dockerfile** - Multi-stage build, non-root user, healthcheck
3. **docker/docker-compose.yml** - Web + Redis services with health checks
4. **requirements.txt** - Dependencies (FastAPI, Uvicorn, pytest, httpx)
5. **docker/.dockerignore** - Excludes venv, cache, env files
6. **tests/** - pytest-compatible test suite

In [ ]:
# Verify project files exist
import os

files_to_check = [
    '../app.py',
    '../docker/Dockerfile',
    '../docker/docker-compose.yml',
    '../docker/.dockerignore',
    '../requirements.txt',
    '../tests/test_smoke.py'
]

print("Project files:")
for f in files_to_check:
    exists = "✓" if os.path.exists(f) else "✗"
    display_name = f.replace('../', '')
    print(f"{exists} {display_name}")

# Expected:
# ✓ app.py
# ✓ docker/Dockerfile
# ✓ docker/docker-compose.yml
# ✓ docker/.dockerignore
# ✓ requirements.txt
# ✓ tests/test_smoke.py

## 3. Dockerfile Walkthrough

### Multi-Stage Build Strategy

```dockerfile
# Stage 1: Builder
FROM python:3.11-slim AS builder
COPY requirements.txt .
RUN pip install --no-cache-dir --user -r requirements.txt

# Stage 2: Runtime
FROM python:3.11-slim
COPY --from=builder /root/.local /root/.local  # Copy only packages
COPY app.py .                                    # Copy app code
RUN useradd -m -u 1000 appuser                  # Non-root user
USER 1000:1000                                   # Switch to non-root
HEALTHCHECK CMD curl -f http://localhost:8000/health || exit 1
CMD ["uvicorn", "app:app", "--host", "0.0.0.0", "--port", "8000"]
```

### Key Optimizations

1. **Multi-stage build**: Separates build dependencies from runtime (smaller final image)
2. **Layer caching**: Copy `requirements.txt` before app code → only reinstall deps when requirements change
3. **Slim base**: `python:3.11-slim` vs `python:3.11` saves 600MB+
4. **Non-root user**: Security best practice (UID 1000)
5. **No cache**: `--no-cache-dir` reduces image size
6. **Health check**: Enables Docker/cloud platform health monitoring
7. **Direct uvicorn**: Use uvicorn directly instead of Python wrapper (faster)

In [ ]:
# Inspect Dockerfile
with open('../docker/Dockerfile', 'r') as f:
    content = f.read()
    lines = content.split('\n')
    print(f"Dockerfile ({len(lines)} lines):\n")
    for i, line in enumerate(lines[:20], 1):  # Show first 20 lines
        print(f"{i:2}  {line}")
    if len(lines) > 20:
        print(f"... ({len(lines) - 20} more lines)")

# Expected:
# Multi-stage Dockerfile with builder and runtime stages

## 4. docker-compose for Local Dev

### Multi-Container Setup

Our `docker-compose.yml` orchestrates:
- **web**: FastAPI application (port 8000)
- **redis**: Cache layer (port 6379)

### Key Features

```yaml
services:
  web:
    build:
      context: ..               # Build from parent directory
      dockerfile: docker/Dockerfile
    ports: ["8000:8000"]
    depends_on:
      redis:
        condition: service_healthy  # Wait for redis to be healthy
    restart: unless-stopped
    networks: [rag-network]
    healthcheck:
      test: ["CMD", "curl", "-f", "http://localhost:8000/health"]
  
  redis:
    image: redis:7-alpine
    volumes: [redis-data:/data]
    networks: [rag-network]
    healthcheck:
      test: ["CMD", "redis-cli", "ping"]  # Redis health check
```

### Benefits

- **One command startup**: `docker compose up -d` (from docker/ directory)
- **Service discovery**: Containers use service names as hostnames
- **Data persistence**: Named volumes survive container deletion
- **Health-based dependencies**: Web waits for redis to be healthy
- **Automatic restarts**: Production-ready failure recovery

In [ ]:
# Show docker-compose structure
import yaml

try:
    with open('../docker/docker-compose.yml', 'r') as f:
        compose = yaml.safe_load(f)
    
    print("Services configured:")
    for service_name, config in compose.get('services', {}).items():
        ports = config.get('ports', [])
        depends_on = config.get('depends_on', {})
        healthcheck = 'healthcheck' in config
        print(f"  • {service_name}: ports={ports}, health={healthcheck}, deps={list(depends_on.keys()) if depends_on else []}")
    
    print(f"\nVolumes: {list(compose.get('volumes', {}).keys())}")
    print(f"Networks: {list(compose.get('networks', {}).keys())}")
except Exception as e:
    print(f"Note: Install PyYAML to parse compose file (pip install pyyaml)")
    print(f"Fallback: Check docker/docker-compose.yml manually")

# Expected:
# Services: web (with health + deps on redis), redis (with health)
# Volumes: redis-data
# Networks: rag-network

## 5. Image Size & Security Notes

### Image Size Comparison

| Base Image | Size | Use Case |
|------------|------|----------|
| `python:3.11` | ~900MB | Development, debugging |
| `python:3.11-slim` | ~180MB | **Production (our choice)** |
| `python:3.11-alpine` | ~50MB | Ultra-minimal (compatibility issues) |

**Our multi-stage image**: ~200-250MB with dependencies (optimized!)

### Security Best Practices Implemented

1. **Multi-stage build**
   - Separates build-time and runtime dependencies
   - Smaller attack surface
   
2. **Non-root user** (`appuser`, UID 1000)
   - Limits damage if container is compromised
   - Standard practice for production

3. **No secrets in image**
   - Environment variables via `.env` file
   - Never COPY `.env` into image

4. **Minimal base image**
   - Less attack surface
   - Fewer dependencies to patch

5. **Health checks**
   - Automatic failure detection
   - Cloud platforms use this for auto-restart

### What's NOT Included (by design)

- No dev tools (reduces size)
- No shell history (security)
- No cache files (size optimization)
- No documentation (see .dockerignore)

In [ ]:
# Estimate image size (requires Docker)
def estimate_image_size():
    try:
        result = subprocess.run(
            ['docker', 'images', '--format', '{{.Repository}}:{{.Tag}} {{.Size}}'],
            capture_output=True,
            text=True,
            timeout=5
        )
        if result.returncode == 0:
            print("Local Docker images:")
            for line in result.stdout.strip().split('\n')[:5]:
                print(f"  {line}")
        else:
            print("No Docker images found (run 'docker build' first)")
    except Exception as e:
        print(f"Docker not available: {e}")
        print("\nExpected base image sizes:")
        print("  python:3.11-slim → ~180MB")
        print("  + FastAPI/Uvicorn → +70MB")
        print("  Total: ~250MB")

# Expected:
# Image size ~250-300MB for our setup
estimate_image_size()

## 6. Alternatives - Deployment Decision Card

### Four Deployment Options

| Option | Best For | Trade-off | Cost |\n|--------|----------|-----------|------|\n| **Docker** | 2+ devs, multi-env | Complexity ↔ Consistency | $10-100/mo |\n| **VMs** | Legacy, OS deps | Isolation ↔ Resource usage | 2-3x Docker |\n| **Bare Metal** | Solo, single server | Performance ↔ Portability | Cheapest |\n| **Kubernetes** | 50+ services | Power ↔ Steep learning curve | $500-5000/mo |

### Decision Framework

**Choose Docker when:**
- ✓ Deploying to multiple environments (dev/staging/prod)
- ✓ Team size 2+ developers
- ✓ Cloud-native deployment
- ✓ Acceptable latency >100ms

**Avoid Docker when:**
- ✗ Solo developer, single server → Use virtualenv + systemd
- ✗ Windows-specific dependencies → Use VMs
- ✗ Ultra-low latency (<5ms) → Bare metal
- ✗ Debugging time >20% on Docker issues → Tool is fighting you

### When NOT to Use Docker

1. **Solo + Single Server**: No environment consistency needed
2. **Windows GUI/COM**: Linux containers won't work
3. **HFT/Real-time**: 10-20% I/O overhead unacceptable

### Full Decision Card

**✅ BENEFIT**: Environment consistency, horizontal scaling, infrastructure as code

**❌ LIMITATION**: Networking complexity, 10-20% I/O overhead, learning curve

**💰 COST**: 4-6 hours learning, $10-100/mo registries, higher infra costs at scale

**🤔 USE WHEN**: Multi-environment, 2+ devs, cloud deployment, >100ms latency OK

**🚫 AVOID WHEN**: Single server, Windows deps, <5ms latency, team unfamiliar with containers

In [ ]:
# Decision helper
def should_use_docker(team_size, num_environments, windows_only, latency_ms):
    \"\"\"Quick decision logic for Docker vs alternatives.\"\"\"\n    
    score = 0\n    reasons = []\n    \n    if team_size >= 2:\n        score += 2\n        reasons.append(\"✓ Team collaboration benefits\")\n    else:\n        reasons.append(\"⚠ Solo dev - consider simpler options\")\n    \n    if num_environments >= 2:\n        score += 3\n        reasons.append(\"✓ Multi-environment consistency critical\")\n    else:\n        reasons.append(\"⚠ Single environment - less Docker benefit\")\n    \n    if windows_only:\n        score -= 5\n        reasons.append(\"✗ Windows-specific - use VMs instead\")\n    \n    if latency_ms < 5:\n        score -= 3\n        reasons.append(\"✗ Ultra-low latency - bare metal better\")\n    elif latency_ms < 100:\n        reasons.append(\"⚠ Performance-sensitive - monitor overhead\")\n    else:\n        score += 1\n        reasons.append(\"✓ Latency tolerance acceptable\")\n    \n    print(\"Docker Decision Score:\", score)\n    for r in reasons:\n        print(f\"  {r}\")\n    \n    if score >= 4:\n        print(\"\\n→ Recommendation: Docker is a good fit\")\n    elif score >= 0:\n        print(\"\\n→ Recommendation: Docker works, but weigh alternatives\")\n    else:\n        print(\"\\n→ Recommendation: Consider alternatives (VMs, bare metal)\")\n    \n    return score\n\n# Example: 3 developers, dev+staging+prod, no Windows, 200ms latency OK\n# Expected: Docker recommended\nshould_use_docker(team_size=3, num_environments=3, windows_only=False, latency_ms=200)

## 7. Troubleshooting - Common Docker Failures

### Failure #1: Port Already in Use

**Error**: `Bind for 0.0.0.0:8000 failed: port is already allocated`

**Fix**:
```bash
# Check what's using the port
lsof -i :8000

# Stop containers properly
docker-compose down

# Or change port mapping
# In docker-compose.yml: \"8001:8000\"
```

---

### Failure #2: Volume Permission Errors

**Error**: `PermissionError: [Errno 13] Permission denied: '/app/data'`

**Fix**:
```bash
# Fix permissions on host
chmod 755 ./data
sudo chown -R 1000:1000 ./data

# Or match user IDs
echo \"USER_ID=$(id -u)\" >> .env
echo \"GROUP_ID=$(id -g)\" >> .env
```

---

### Failure #3: Container Networking Issues

**Error**: `ConnectionError: Error connecting to redis:6379`

**Fix**:
```bash
# Verify network
docker network inspect rag-network

# Test connectivity
docker-compose exec web ping redis

# Use service names, not localhost!
# REDIS_HOST=redis  ✓ Correct
# REDIS_HOST=localhost  ✗ Wrong
```

---

### Failure #4: Stale Image Cache

**Error**: Code changes don't appear in container

**Fix**:
```bash
# Force rebuild
docker-compose build --no-cache

# Clean everything
docker-compose down -v
docker system prune -a
```

---

### Failure #5: Out of Memory (Exit 137)

**Error**: Container exits with code 137

**Fix**:
```bash
# Monitor resources
docker stats

# Increase memory limit in docker-compose.yml:
# deploy:
#   resources:
#     limits:
#       memory: 4G
```

---

### Debug Checklist

1. **Check logs**: `docker-compose logs -f`
2. **Inspect container**: `docker inspect <container>`
3. **Get inside**: `docker-compose exec web /bin/bash`
4. **Check resources**: `docker stats`
5. **Verify network**: `docker network inspect rag-network`

In [ ]:
# Quick Docker commands reference
print("Common Docker Commands:")
print()
print("Build & Run (from docker/ directory):")
print("  cd docker")
print("  docker compose build")
print("  docker compose up -d")
print("  docker compose down")
print()
print("Debug:")
print("  docker compose logs -f web")
print("  docker compose ps")
print("  docker stats")
print("  docker compose exec web /bin/bash")
print()
print("Clean:")
print("  docker compose down -v  # Remove volumes")
print("  docker system prune -a  # Clean all unused")
print()
print("Health:")
print("  curl http://localhost:8000/health")
print("  python app.py --health  # From project root")
print()
print("Tests:")
print("  pytest  # From project root")
print()
print("For full examples, see README.md")

# Expected:
# Command reference printed

---

## Summary & Next Steps

### What We Built

✓ FastAPI application with health endpoints (app.py)
✓ Multi-stage Dockerfile (optimized, secure)
✓ docker-compose with health-based dependencies
✓ Redis cache layer with persistence
✓ pytest-compatible test suite
✓ TVH standard per-module layout

### Key Takeaways

1. **Multi-stage builds** reduce image size and attack surface
2. **Health checks** enable automatic recovery and dependency ordering
3. **Non-root users** are security best practice
4. **Layer caching matters** - copy requirements before code
5. **Service discovery** via Docker networks (use service names, not localhost)
6. **depends_on with condition** ensures proper startup order

### Challenges

**🟢 EASY**: Add PostgreSQL container for query logging
**🟡 MEDIUM**: Implement production vs development docker-compose profiles
**🔴 HARD**: Add nginx reverse proxy with SSL termination

### Next Steps

1. Run tests: `pytest` from project root
2. Deploy to Railway/Render (M3.2)
3. API Development & Security (M3.3)
4. Load Testing & Scaling (M3.4)

### Resources

- README.md - Full quickstart guide with learning arc
- tests/test_smoke.py - FastAPI TestClient examples
- Docker docs: https://docs.docker.com

---

**End of M3.1 Notebook**